This notebook is used for every view and sex. Just datapath is changed.

In [ ]:
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from PIL import Image
from tensorflow.keras import backend as K

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Path
DATA_PATH = '/content/drive/MyDrive/M'
BATCH_SIZE = 32

In [ ]:
# Load the dataset
def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [224, 224])
    img = (img - 127.5) / 127.5  # Normalize to [-1,1]
    return img

In [ ]:
# Create a TensorFlow dataset
dataset = tf.data.Dataset.list_files(DATA_PATH + '/*.jpg')
dataset = dataset.map(load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset = dataset.shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
from tensorflow.keras.layers import Softmax

In [ ]:
# self attention
class SelfAttention(keras.layers.Layer):
    def __init__(self, in_dim, attention_dim):
        super(SelfAttention, self).__init__()
        self.query_conv = layers.Conv2D(filters=attention_dim, kernel_size=1, padding='same')
        self.key_conv = layers.Conv2D(filters=attention_dim, kernel_size=1, padding='same')
        self.value_conv = layers.Conv2D(filters=in_dim, kernel_size=1, padding='same')
        self.gamma = self.add_weight(name='gamma', shape=(), initializer='zeros', trainable=True)
        self.softmax = layers.Softmax(axis=-1)

    def call(self, x):
        # Accessing the shape components by indexing
        batch_size = tf.shape(x)[0]
        height = tf.shape(x)[1]
        width = tf.shape(x)[2]
        C = tf.shape(x)[3]

        proj_query = tf.reshape(self.query_conv(x), [batch_size, -1, width*height])
        proj_query = tf.transpose(proj_query, perm=[0, 2, 1])
        proj_key = tf.reshape(self.key_conv(x), [batch_size, -1, width*height])
        energy = tf.matmul(proj_query, proj_key)
        attention = self.softmax(energy)
        proj_value = tf.reshape(self.value_conv(x), [batch_size, -1, width*height])

        out = tf.matmul(proj_value, tf.transpose(attention, perm=[0, 2, 1]))
        out = tf.reshape(out, [batch_size, height, width, C])
        out = self.gamma * out + x
        return out


In [ ]:
def make_generator_model(input_shape=(100,)):
    model = keras.Sequential()

    # Starting with a 7x7x256 tensor
    model.add(layers.Dense(7*7*256, use_bias=False, input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Reshape((7, 7, 256)))

    # Upscale to 14x14
    model.add(layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))

    # Upscale to 28x28
    model.add(layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))

    # Add Self-Attention here
    model.add(SelfAttention(64, 16))

    # Upscale to 56x56
    model.add(layers.Conv2DTranspose(32, kernel_size=4, strides=2, padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))

    # Upscale to 112x112
    model.add(layers.Conv2DTranspose(16, kernel_size=4, strides=2, padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))

    # Upscale to 224x224
    model.add(layers.Conv2DTranspose(3, kernel_size=4, strides=2, padding='same', use_bias=False, activation='tanh'))

    return model

In [ ]:
def make_discriminator_model(img_shape=(224, 224, 3)):
    model = keras.Sequential()

    model.add(layers.Conv2D(16, kernel_size=4, strides=2, padding='same', input_shape=img_shape))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.5))

    model.add(layers.Conv2D(32, kernel_size=4, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.5))

    model.add(layers.Conv2D(64, kernel_size=4, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.5))

    model.add(layers.Conv2D(128, kernel_size=4, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.5))

    model.add(layers.Conv2D(256, kernel_size=4, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.5))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model

In [ ]:
generator = make_generator_model()
discriminator = make_discriminator_model()

In [ ]:
# Loss and optimizers
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

initial_learning_rate = 0.0002

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=1000,
    decay_rate=0.96,
    staircase=True)

# Adjusted the beta_1 value for the Adam optimizer
generator_optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule, beta_1=0.5)

In [ ]:
# Seed for consistent image generation
seed = tf.random.normal([150, 100])

In [ ]:
@tf.function
def train_step(images):
    current_batch_size = images.shape[0]  # Get the current batch size
    noise = tf.random.normal([current_batch_size, 100])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        # Use the real_labels_smoothed and fake_labels_smoothed approach or the original one
        real_labels_smoothed = tf.ones_like(real_output) * 0.9
        fake_labels_smoothed = tf.zeros_like(fake_output) + 0.1

        gen_loss = cross_entropy(tf.ones_like(fake_output), fake_output)
        real_loss = cross_entropy(real_labels_smoothed, real_output)
        fake_loss = cross_entropy(fake_labels_smoothed, fake_output)
        disc_loss = real_loss + fake_loss

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)


    # Clipping the gradients
    clipped_gen_gradients = [tf.clip_by_value(grad, -1., 1.) for grad in gradients_of_generator]
    clipped_disc_gradients = [tf.clip_by_value(grad, -1., 1.) for grad in gradients_of_discriminator]

    generator_optimizer.apply_gradients(zip(clipped_gen_gradients, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(clipped_disc_gradients, discriminator.trainable_variables))

    return gen_loss, disc_loss

In [ ]:
def generate_and_save_images(model, epoch, test_input):
    # Ensure output path exists
    output_path = os.path.join(DATA_PATH, 'output_GAN')
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    print("Generating predictions...")
    predictions = model(test_input, training=False)
    print("Converting predictions...")
    predictions = (predictions + 1) * 127.5  # Convert from [-1, 1] to [0, 255]
    predictions = tf.cast(predictions, tf.uint8)
    print("Saving images...")

    for i, img in enumerate(predictions):
        path = os.path.join(output_path, f'image_epoch{epoch}_sample{i}.png')
        tf.keras.preprocessing.image.save_img(path, img)

In [ ]:
# Training loop
def train(dataset, epochs):
  for epoch in range(epochs):
    gen_loss_metric = tf.keras.metrics.Mean()
    disc_loss_metric = tf.keras.metrics.Mean()

    for images in dataset:
        gen_loss, disc_loss = train_step(images)
        gen_loss_metric(gen_loss)
        disc_loss_metric(disc_loss)

    # Print the losses
    print(f"Epoch {epoch+1}, Generator Loss: {gen_loss_metric.result()}, Discriminator Loss: {disc_loss_metric.result()}")

    # Generate and save images only for the last 10 epochs
    if epoch >= (epochs - 5):
        generate_and_save_images(generator, epoch + 1, seed)

In [ ]:
num_epochs = 10000  # Adjust as needed
train(dataset, num_epochs)